<a href="https://colab.research.google.com/github/menouahmad/bonus-III/blob/main/Copy_of_bonus_gradio_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Huggingface and gradio

A different library for sharing applications called `gradio` is becoming popular with easy deep learning model integrations.  For this assignment, your goal is to write a basic tutorial that:

- Discusses the basics of building a `gradio` app with rows, columns, images, input fields, and sliders.  Demonstrate this with a `gradio` app of your imagination (different than docs creativity matters!).
- Demonstrates how to use the huggingface `transformers` library and its pipelines to build a `gradio` app. [here](https://www.gradio.app/guides/using-hugging-face-integrations)

Create a github repository with your tutorial notebook and a README with a paragraph overview of the tutorial and description of huggingface model and app.

In [ ]:
import gradio as gr
import numpy as np
from PIL import Image
from transformers import pipeline

In [ ]:
##Creative Gradio App
# Function to apply a "mood" tint to an image and generate a caption
def mood_modifier(image, user_name, intensity):
    if image is None:
        return None, "Please upload an image first!"

    # Gradio passes images as numpy arrays by default
    # We make a copy to avoid modifying the original directly
    modified_img = np.copy(image)

    # Adjust the red channel based on the slider intensity (0-100)
    # This creates a "warm" or "angry" red tint effect
    factor = intensity / 100.0
    modified_img[:, :, 0] = np.clip(modified_img[:, :, 0] * (1 + factor), 0, 255)

    # Generate a personalized message
    caption = f"Here is {user_name}'s photo with a mood intensity of {intensity}%!"

    return modified_img, caption

# Create the Gradio application using Blocks for custom layout
with gr.Blocks() as demo_app:
    gr.Markdown("# 🎨 The Photo Mood Adjuster")
    gr.Markdown("Upload a photo, enter your name, and use the slider to set the 'Warmth' intensity.")

    # Use Rows and Columns to organize the layout
    with gr.Row():
        # Left Column: Inputs
        with gr.Column():
            img_input = gr.Image(label="Upload Image")
            text_input = gr.Textbox(label="Your Name", placeholder="Enter name here...")
            slider_input = gr.Slider(minimum=0, maximum=100, value=20, label="Mood Intensity")
            submit_btn = gr.Button("Apply Filter")

        # Right Column: Outputs
        with gr.Column():
            img_output = gr.Image(label="Processed Image")
            text_output = gr.Textbox(label="Result Caption")

    # Connect the inputs and outputs to the function
    submit_btn.click(
        fn=mood_modifier,
        inputs=[img_input, text_input, slider_input],
        outputs=[img_output, text_output]
    )

# Launch the app
demo_app.launch()

In [ ]:
#Hugging Face Transformers Integration

# Load a pre-trained sentiment analysis pipeline
# This downloads a model capable of classifying text as POSITIVE or NEGATIVE
classifier = pipeline("sentiment-analysis")

# Define a simple wrapper function for the model
def analyze_text(text):
    results = classifier(text)
    # Extract label and score to make it readable
    label = results[0]['label']
    score = results[0]['score']
    return f"Sentiment: {label} (Confidence: {score:.2f})"

# Create a Gradio Interface
# This is the quickest way to wrap a function
hf_demo = gr.Interface(
    fn=analyze_text,
    inputs=gr.Textbox(lines=2, placeholder="Type something like 'I love coding!'..."),
    outputs="text",
    title="Hugging Face Sentiment Analyzer",
    description="Enter any sentence, and the AI will determine if the sentiment is Positive or Negative."
)

# Launch the interface
hf_demo.launch()